In [1]:
import geemap
import ee
import os
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
os.chdir(r'E:\Work\QTPofShp')

In [ ]:
from glob import glob

In [ ]:
year = 1980
images = ee.ImageCollection("ECMWF/ERA5/MONTHLY")\
            .filterDate(str(year), str(year+1))